In [3]:
from scipy import sparse
import numpy as np

In [4]:
Id = np.identity(2)
j =1j
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-j],[j,0]])
Z = np.array([[1,0],[0,-1]])
S = np.array([[1,0],[0,j]])
Sd = np.array([[1,0],[0,-j]])
T = np.array([[1,0],[0,np.exp(j*(np.pi/4))]])
Td = np.array([[1,0],[0,np.exp(-j*(np.pi/4))]])
h = (1/2**0.5)*np.array([[1,1],[1,-1]])

In [5]:
#Initial_Qubit

In [6]:
def single_q(i):
    if i == 0:
        return np.array([1,0])
    elif i==1:
        return np.array([0,1])
    else:
        return "NULL"
P0 = np.outer(single_q(0),single_q(0))
P1 = np.outer(single_q(1),single_q(1))

In [7]:
def init_q(qubit):
    q = single_q(qubit[0])
    for i in range(1,len(qubit)):
        q = sparse.kron(q,single_q(qubit[i]))
    iq = q.toarray()
    return iq.T
'''
def init_reverse(qubit):
    for i in range(len(qubit)):
        if float(qubit[i].real) > 1e-5:
            f = bin(i).replace('0b','')
    while(len(f) != int(np.log2(len(qubit)))):
        f = "0"+ f
    arr = []
    for i in f:
        arr.append(int(i))
    return arr
'''
def tobinary(k,n):
    f = bin(k).replace('0b','')
    while(len(f) < n):
        f = "0"+ f
    arr = []
    for i in f:
        arr.append(int(i))
    return arr

In [8]:
#Single_Qubit_Gate

In [9]:
def Hadmard_Gate(hl):#Location of Hadamard Gate:Input array 
#for example:if Hadamard gate is [0,0,0,1] which means the location of the Hadamard Gate is on the fourth qubit
    h = (1/2**0.5)*np.array([[1,1],[1,-1]])
    H = h*hl[0]+Id*(not hl[0])
    for i in range(1,len(hl)):
        H = sparse.kron(H,h*hl[i]+Id*(not hl[i]))
    return H.toarray()

In [10]:
def Single_Gate(gl,Gate):#Location of Single_Gate:Input array 
#for example:if Hadamard gate is [0,0,0,1] which means the location of the Hadamard Gate is on the fourth qubit
    gate = Gate*gl[0]+Id*(not gl[0])
    for i in range(1,len(gl)):
        gate = sparse.kron(gate,Gate*gl[i]+Id*(not gl[i]))
    return gate.toarray()

In [11]:
def Rx_Gate(theta):
    return np.cos(theta/2)*Id-j*np.sin(theta/2)*X
def Ry_Gate(theta):
    return np.cos(theta/2)*Id-j*np.sin(theta/2)*Y
def Rz_Gate(theta):
    return np.cos(theta/2)*Id-j*np.sin(theta/2)*Z

In [12]:
#Z-Y-Z Decomposition

In [13]:
#Multibit_Gate

In [14]:
def CNOT_Gate(cl,tl):
#cl:Location of Control qubit Gate:Input array 
#tl:Location of Target qubit Gate:Input array
    CNOT1 = P0*cl[0]+Id*(not cl[0])
    for i in range(1,len(cl)):
        CNOT1 = np.kron(CNOT1,P0*cl[i]+Id*(not cl[i]))
    CNOT2 = P1*cl[0]+Id*(not cl[0])+ X*(tl[0])
    for i in range(1,len(cl)):
        CNOT2 = np.kron(CNOT2,P1*cl[i]+Id*((not cl[i])and(not tl[i]))+ X*(tl[i]))
    return CNOT1+CNOT2

In [15]:
#We can use the above gate to construct any algorithm you want

In [16]:
#for example:

In [17]:
def measure(ml,tf,q):#the location of measure
    ls = []
    n = len(ml)
    for i in range(2**len(ml)):
        ls.append(tobinary(i,len(ml)))
    qs = []
    m = []
    dic = {}
    result = []
    for j in range(2**len(ml)):
        qs.append(single_q(ls[j][0]))
    for j in range(2**len(ml)):
        for i in range(1,n):
            qs[j] = np.kron(qs[j],single_q(ls[j][i]))
    for k in range(len(qs)):
        dic[f'{tobinary(k,n)}'] = np.dot(qs[k],q)[0]
        m.append(np.dot(qs[k],q)[0])
        result.append(tobinary(k,n))
    for i in range(len(ml)):
        if ml[i]!=0:
            ord = i
    prob = 0
    for i in result:
        if i[ord] == tf:
            prob += dic[f"{i}"]**2
    return prob

In [18]:
import random
def rand_pick(seq,prob):
    x = random.uniform(0,1)
    cumprob = 0.0
    for item,item_prob in zip(seq,prob):
        cumprob += item_prob
        if x<cumprob:
            break
    return item
def real_measure(ml,q):
    seq = [0,1]
    prob = [measure(ml,0,q),measure(ml,1,q)]
    return rand_pick(seq,prob)

In [19]:
def ccnotgate(qubit):
    q = init_q(qubit)
    q = Hadmard_Gate([0,0,1])@q
    q = CNOT_Gate([0,1,0],[0,0,1])@q
    q = Single_Gate([0,0,1],Td)@q
    q = CNOT_Gate([1,0,0],[0,0,1])@q
    q = Single_Gate([0,0,1],T)@q
    q = CNOT_Gate([0,1,0],[0,0,1])@q
    q = Single_Gate([0,0,1],Td)@q
    q = CNOT_Gate([1,0,0],[0,0,1])@q
    q = Single_Gate([0,1,0],Td)@q
    q = Single_Gate([0,0,1],T)@q
    q = CNOT_Gate([1,0,0],[0,1,0])@q
    q = Hadmard_Gate([0,0,1])@q
    q = Single_Gate([0,1,0],Td)@q
    q = CNOT_Gate([1,0,0],[0,1,0])@q
    q = Single_Gate([1,0,0],T)@q
    q = Single_Gate([0,1,0],S)@q
    return q

In [20]:
init_reverse(ccnotgate([1,1,0]))

NameError: name 'init_reverse' is not defined

In [21]:
q = init_q([0,1])
q = Hadmard_Gate([1,1])@q
q = Hadmard_Gate([1,0])@q
measure([1,0],0,q)

0.9999999999999996

In [39]:
def CR_Gate(cl,tl,Gate):
#cl:Location of Control qubit Gate:Input array 
#tl:Location of Target qubit Gate:Input array
    CR1 = P0*cl[0]+Id*(not cl[0])
    for i in range(1,len(cl)):
        CR1 = np.kron(CR1,P0*cl[i]+Id*(not cl[i]))
    CR2 = P1*cl[0]+Id*(not cl[0])+ Gate*(tl[0])
    for i in range(1,len(cl)):
        CR2 = np.kron(CR2,P1*cl[i]+Id*((not cl[i])and(not tl[i]))+ Gate*(tl[i]))
    return CR1+CR2

In [40]:
CNOT = CNOT_Gate([1,0],[0,1])

In [50]:
def R(n):
    f = np.array([[1,0],[0,np.exp(2*np.pi*j/2**n)]])
    return f

In [51]:
def CZ():
    CZ = Hadmard_Gate([0,1])
    CZ = np.dot(CNOT_Gate([1,0],[0,1]),CZ)
    CZ = np.dot(Hadmard_Gate([0,1]),CZ)
    return CZ

In [54]:
print(R(0))

[[1.+0.0000000e+00j 0.+0.0000000e+00j]
 [0.+0.0000000e+00j 1.-2.4492936e-16j]]


In [57]:
print(CR_Gate([1,0],[0,1],R(3)))

[[1.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         1.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         1.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.70710678+0.70710678j]]


In [59]:
#Three Qubit Fourier Transform
def threequbitfft():
    fft = Hadmard_Gate([0,0,1])
    fft = np.dot(CR_Gate([0,1,0],[0,0,1],R(2)),fft)
    fft = np.dot(CR_Gate([0,1,0],[0,0,1],R(2)),fft)
    fft = np.dot(CR_Gate([1,0,0],[0,0,1],R(3)),fft)
    fft = np.dot(Hadmard_Gate([0,1,0]),fft)
    fft = np.dot(CR_Gate([1,0,0],[0,1,0],R(2)),fft)
    fft = np.dot(Hadmard_Gate([1,0,0]),fft)
    return fft
print(threequbitfft())

[[ 3.53553391e-01+0.00000000e+00j  3.53553391e-01+0.00000000e+00j
   3.53553391e-01+0.00000000e+00j  3.53553391e-01+0.00000000e+00j
   3.53553391e-01+0.00000000e+00j  3.53553391e-01+0.00000000e+00j
   3.53553391e-01+0.00000000e+00j  3.53553391e-01+0.00000000e+00j]
 [ 3.53553391e-01+0.00000000e+00j -3.53553391e-01+0.00000000e+00j
  -3.53553391e-01+4.32978028e-17j  3.53553391e-01-4.32978028e-17j
   2.50000000e-01+2.50000000e-01j -2.50000000e-01-2.50000000e-01j
  -2.50000000e-01-2.50000000e-01j  2.50000000e-01+2.50000000e-01j]
 [ 3.53553391e-01+0.00000000e+00j  3.53553391e-01+0.00000000e+00j
  -3.53553391e-01+0.00000000e+00j -3.53553391e-01+0.00000000e+00j
   2.16489014e-17+3.53553391e-01j  2.16489014e-17+3.53553391e-01j
  -2.16489014e-17-3.53553391e-01j -2.16489014e-17-3.53553391e-01j]
 [ 3.53553391e-01+0.00000000e+00j -3.53553391e-01+0.00000000e+00j
   3.53553391e-01-4.32978028e-17j -3.53553391e-01+4.32978028e-17j
  -2.50000000e-01+2.50000000e-01j  2.50000000e-01-2.50000000e-01j
  -2.50